In [16]:
import json
import re
import os
import ast
import csv
import pandas as pd

In [17]:
def decode_utf8(item):
    if isinstance(item, str):  # If item is a string
        try:
            return item.encode('latin1').decode('utf-8')
        except:
            return item  # Return original if decoding fails
    elif isinstance(item, list):  # If item is a list
        return [decode_utf8(x) for x in item]
    elif isinstance(item, dict):  # If item is a dictionary
        return {key: decode_utf8(value) for key, value in item.items()}
    else:  # If item is neither string, list nor dict (could be int, bool, etc.)
        return item

In [18]:
def clean_n_decode(data_in, file_name):
    splited_data = data_in.replace("b'{", 'b"{').split('b"')
    arr_data = [spld.replace("\\'", "'") for spld in splited_data]

    json_data_arr = []
    
    for i in range(len(arr_data)):
        if i == 0:
            continue
        elif "TikTokApi.video(" in arr_data[i]:
            continue
        else:
            try:
                # clean & decode UTF-8
                content = arr_data[i].replace("'", '"')[:-2]
                content = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", content) # reg replace from Don"t to Don't

                # # analyze error
                # pattern1 = r'"title":\s*"(.*?)"'
                # pattern2 = r'"desc":\s*"(.*?)"'
                # updated_string = re.sub(pattern1, '"title": "ignore string error"', content)
                # updated_string = re.sub(pattern2, '"desc": "ignore string error"', updated_string).replace("True", "true").replace("False", "false")

                str_data = ast.literal_eval(content)
                decoded_data = str(decode_utf8(str_data)).replace("'", '"').replace("True", "true").replace("False", "false").replace("\\\\n", " ")
                updated_string = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", decoded_data) # reg replace from Don"t to Don't
                # print(updated_string) # print and paste this on the .json file to see if there's an error

                # turn string into object
                json_data = json.loads(updated_string)
                id = json_data["id"]

                # print(id, json_data)

                # # export to check json data
                # file_path = r"./exported-accountpage-json-error-check/" + file_name[:-4] + "-" + id + ".json"
                # with open(file_path, "w", encoding='utf-8') as file:
                #     json.dump(json_data, file, indent=4, ensure_ascii=False)
                
                
                json_data_arr.append(json_data)
                break # for one time loop
            except:
                print(file_name + ' error inside function')
                break
    return json_data_arr


In [19]:
def extract_nested_keys(my_dict):

    keys_to_extract = [
        ["author", "uniqueId"],
        ["author", "signature"],
        ["author", "verified"],
        "desc",
        "verified",
        "createTime",
        "digged",
        "duetDisplay",
        "duetEnabled",
        "forFriend",
        "id",
        "itemCommentStatus",
        ["music", "authorName"],
        ["music", "coverLarge"],
        ["music", "duration"],
        ["music", "id"],
        ["music", "original"],
        ["music", "playUrl"],
        ["music", "title"],
    ]

    result = {}
    for key in keys_to_extract:
        if isinstance(key, list):
            nested_result = my_dict
            for nested_key in key:
                if nested_key in nested_result:
                    nested_result = nested_result[nested_key]
                    # print(nested_key)
                else:
                    nested_result = None
                    break
            if nested_result is not None:
                result[str("_".join(key))] = nested_result
        elif key in my_dict:
            result[key] = my_dict[key]
    return result



In [20]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

        
    # print(hashtag_arr)
    return hashtag_arr

In [21]:
def extract_data(json_data):
    extracted_values = extract_nested_keys(json_data)
    extracted_values["hashtagNames"] = extract_hashtags(json_data["contents"])
    print(str(extracted_values).replace("'",'"').replace("True", "true").replace("False", "false"))
    return extracted_values


In [27]:
origin_folder_path = r"./txt-accounts-files"

with open("AccountPages.csv", "a", newline="", encoding="utf-8") as csvfile:
    csvfile.write("")
    
    for i, file_name in enumerate(os.listdir(origin_folder_path)):
        if file_name.endswith('.txt'):
            file_path = os.path.join(origin_folder_path, file_name)

            try:
                with open(file_path, "r") as file:
                    file_data = file.read()
                
                json_data_arr = clean_n_decode(file_data, file_name)
                extracted_df = pd.DataFrame([extract_data(json_data) for json_data in json_data_arr])
                
                # header = list(extracted_data_in_json_arr[0].keys())

                # # append data to csv
                # writer = csv.DictWriter(csvfile, fieldnames=header)
                # if csvfile.tell() == 0:
                #     writer.writeheader()
                # # writer.writerow(extracted_data_json)
                # for row in extracted_data_in_json_arr:
                #     writer.writerow({k: row[k] if k in row else "x1x" for k in header})

            except:
                print(file_name + ' error')

{"author_uniqueId": "1997thantzinoo1", "author_signature": "FB = Thant Zin Oo page = 1997kothantzin Line = 1997thant", "author_verified": false, "desc": "@Win Thu Aung ", "createTime": 1697677280, "digged": false, "duetDisplay": 0, "duetEnabled": true, "forFriend": false, "id": "7291468383554407685", "itemCommentStatus": 0, "music_authorName": "🖤MyoThiHaKyaw🖤", "music_coverLarge": "https://p16-sign-sg.tiktokcdn.com/aweme/1080x1080/tos-alisg-avt-0068/9ff17c629d71d6480b63771f78abb58e.jpeg?x-expires=1698152400&x-signature=skD%2BfuGN80EvzgHKrIKyVbvuPYQ%3D", "music_duration": 23, "music_id": "7260431444898581250", "music_original": false, "music_playUrl": "https://v16-webapp-prime.tiktok.com/video/tos/alisg/tos-alisg-v-27dcd7/oY2Q7OiACFG4fewgWwWtFmJgHIfAeTAVPQmIHM/?a=1988&ch=0&cr=0&dr=0&er=0&lr=default&cd=0%7C0%7C0%7C0&br=250&bt=125&bti=ODszNWYuMDE6&ft=t2PAl-Inz7T_iIFZiyq8Z&mime_type=audio_mpeg&qs=6&rc=ZjxlaTNoOjs4Zzk3ZTRmM0BpMzR5ajM6ZnY2bTMzODU8NEA0NjVgXzAuXy8xYzAwLV9eYSNyX2locjRfay5gLS1kM